# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686519


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<00:59,  2.14s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:01,  2.27s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.72s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.26s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  3.78it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  3.94it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.12it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.30it/s]

not-logged-in-20a275b53abec8db9f41     0.007132
RockyGeo                               0.000483
Slartibartfast11                       0.034882
shirogane                              0.000637
AlexMercer                             0.002102
not-logged-in-a9247261537fa1da33fe     0.005187
kjkavene                               0.000694
mtajnai                                0.000934
bldelacr                               0.017208
justsomenoodles                        0.000515
AdamMorse                              0.000417
jmartinez468                           8.490332
not-logged-in-9f30f411df9d28d2c541     0.002620
Lavadude                               0.004311
Mr_DMan                                0.002306
galesofnovember                        0.003920
not-logged-in-954c0e347f1d0c012256     0.002660
not-logged-in-670efeb17b0d41814f58     0.001655
not-logged-in-6b5b181f87c068d26d7b     0.016461
Mkcls                                  0.002413
Fazal.H                                0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())